In [5]:
# path to the model weights files.
weights_path = 'weights/vgg16_weights.h5'
top_model_weights_path = 'fc_model.h5'
train_data_dir = 'data2/train/attempt1'
validation_data_dir = 'data2/validation/attempt1'
nb_train_samples = 870
nb_validation_samples = 260
nb_epoch = 10

# dimensions of our images.
img_width, img_height = 224,224

In [8]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import numpy as np
import os
import h5py

In [7]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(3,img_width, img_height)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

In [9]:
model.load_weights(weights_path)

In [43]:
import pydot
import graphviz

def plot(model, to_file='model.png'):                                           

    graph = pydot.Dot(graph_type='digraph')                                     
    if type(model) == Sequential:                                                                                                              
        previous_node = None                                                                                                                   
        written_nodes = []                                                      
        n = 1                                                                                                                                  
        for layer in model.layers:                                              
            # append number in case layers have same name to differentiate      
            config = layer.get_config()                                         
            if (config['name'] + str(n)) in written_nodes:                      
                n += 1                                                          
            current_node = pydot.Node(config['name'] + str(n))                  
            written_nodes.append(config['name'] + str(n))                       
            graph.add_node(current_node)                                            
            if previous_node:                                                   
                graph.add_edge(pydot.Edge(previous_node, current_node))             
            previous_node = current_node                                                
        graph.write_png(to_file)                                                    

    elif type(model) == Graph:                                                      
        config = model.get_config()                                                 
        # don't need to append number for names since all nodes labeled                 
        for input_node in config['input_config']:                                   
            graph.add_node(pydot.Node(input_node['name']))                      

        # intermediate and output nodes have input defined                      
        for layer_config in [config['node_config'], config['output_config']]:   
            for node in layer_config:                                           
                graph.add_node(pydot.Node(node['name']))                        
                # possible to have multiple 'inputs' vs 1 'input'               
                if node['inputs']:                                              
                    for e in node['inputs']:                                    
                        graph.add_edge(pydot.Edge(e, node['name']))             
                else:                                                           
                    graph.add_edge(pydot.Edge(node['input'], node['name']))     

        graph.write_png(to_file)  

In [64]:
plot(model)

In [63]:
print model.layers[-1]
print model.output_shape

(None, 3)


In [28]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

In [29]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

In [30]:
model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [49]:
im = preprocess_image_batch(['data2/nike-1.jpg'],img_size=(256,256), crop_size=(img_width, img_height), color_mode="bgr")

In [50]:
from scipy.misc import imread, imresize, imsave
im = preprocess_image_batch(['data2/nike-1.jpg','data2/nike-2.jpg','data2/nike-3.jpg','data2/nike-4.jpg','data2/nike-5.jpg','data2/nike-6.jpg'],img_size=(256,256), crop_size=(img_width, img_height), color_mode="bgr")

In [51]:
out = model.predict(im)
print np.argmax(out)

2


In [52]:
# Show top 5 predictions
np.argsort(out)[::-1][:5]

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2]])

In [55]:
from keras.preprocessing.image import ImageDataGenerator

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [56]:
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

Found 870 images belonging to 3 classes.


In [57]:
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

Found 260 images belonging to 3 classes.


In [59]:
print nb_validation_samples

260


In [58]:
# fine-tune the model
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

Epoch 1/10


Exception: Error when checking model target: expected dense_8 to have shape (None, 3) but got array with shape (32, 1)

In [13]:
from scipy.misc import imread, imresize, imsave

def preprocess_image_batch(image_paths, img_size=None, crop_size=None, color_mode="rgb", out=None):
    img_list = []
    
    for im_path in image_paths:
        img = imread(im_path, mode='RGB')
        if img_size:
            img = imresize(img,img_size)
            
        img = img.astype('float32')
        # We permute the colors to get them in the BGR order
        if color_mode=="bgr":
            img[:,:,[0,1,2]] = img[:,:,[2,1,0]]
        # We normalize the colors with the empirical means on the training set
        img[:, :, 0] -= 123.68 
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 103.939
        img = img.transpose((2, 0, 1))

        if crop_size:
            img = img[:,(img_size[0]-crop_size[0])//2:(img_size[0]+crop_size[0])//2
                      ,(img_size[1]-crop_size[1])//2:(img_size[1]+crop_size[1])//2]
            
        img_list.append(img)

    img_batch = np.stack(img_list, axis=0)
    if not out is None:
        out.append(img_batch)
    else:
        return img_batch